# L.A. County places timeseries

In [1]:
# !/opt/anaconda3/bin/python -m pip install nb-black

In [2]:
%load_ext lab_black

In [3]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt

In [4]:
%matplotlib inline
import json
import numpy as np
import altair as alt
import altair_latimes as lat

In [5]:
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
from datetime import datetime, date

In [6]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

In [7]:
surge_begin = "2020-11-15"
surge_end = "2021-01-15"

### Fetch cases timeseries from LAT coronavirus tracker repo

In [8]:
cases = pd.read_csv(
    "../../coronavirus-tracker/_notebooks/data/processed/places/timeseries.csv",
    low_memory=False,
)

In [82]:
cases[cases["date"] == "2020-12-31"]

,name,date,county,fips,slug,confirmed_cases_note,confirmed_cases,lat,lon,population,zcta_id,id,new_confirmed_cases,new_confirmed_cases_seven_day_total,new_confirmed_cases_seven_day_average,new_confirmed_cases_fourteen_day_total,new_confirmed_cases_fourteen_day_average,confirmed_cases_per_100k,new_confirmed_cases_per_100k,new_confirmed_cases_seven_day_per_100k,new_confirmed_cases_fourteen_day_per_100k
307,94501: Alameda,2020-12-31,Alameda,1,94501-alameda,NaN,1038,37.774606,-122.274583,62826.0,94501.0,94501,39.0,115.0,16.428571,258.0,18.428571,1652.182218,62.076210,183.045236,410.658008
662,94502: Alameda,2020-12-31,Alameda,1,94502-alameda,NaN,145,37.736988,-122.241149,14117.0,94502.0,94502,11.0,19.0,2.714286,36.0,2.571429,1027.130410,77.920238,134.589502,255.011688
1017,94505: Discovery Bay,2020-12-31,Alameda,1,94505-discovery-bay,NaN,0,37.882787,-121.593154,16382.0,94505.0,94505,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1372,94536: Fremont,2020-12-31,Alameda,1,94536-fremont,NaN,1631,37.570977,-121.987951,72976.0,94536.0,94536,43.0,171.0,24.428571,396.0,28.285714,2234.981364,58.923482,234.323613,542.644157
1727,94538: Fremont,2020-12-31,Alameda,1,94538-fremont,NaN,1668,37.499148,-121.977924,63911.0,94538.0,94538,51.0,186.0,26.571429,416.0,29.714286,2609.879363,79.798470,291.029713,650.905165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401202,Marysville,2020-12-31,Yuba,115,marysville,NaN,1665,39.150000,-121.583333,12282.0,NaN,Marysville,0.0,205.0,29.285714,403.0,28.785714,13556.424035,0.000000,1669.109266,3281.224556
401415,Olivehurst Linda,2020-12-31,Yuba,115,olivehurst-linda,NaN,1112,39.127778,-121.550833,32623.0,NaN,Olivehurst Linda,0.0,93.0,13.285714,222.0,15.857143,3408.638077,0.000000,285.074947,680.501487
401628,Other,2020-12-31,Yuba,115,other,NaN,227,NaN,NaN,NaN,NaN,Other,0.0,14.0,2.000000,32.0,2.285714,NaN,NaN,NaN,NaN
401841,Plumas Lake,2020-12-31,Yuba,115,plumas-lake,NaN,396,39.020684,-121.557993,7367.0,NaN,Plumas Lake,0.0,27.0,3.857143,89.0,6.357143,5375.322384,0.000000,366.499253,1208.090132


### Isolate those places in Los Angeles County

In [9]:
places = cases[(cases["zcta_id"].isna()) & (cases["county"] == "Los Angeles")]

### Limit the dataframe to places with > 3000 population

In [10]:
places_lg = places[places["population"] >= 3000]

---

### Get neightborhoods and regions in LA County from U.S. Census

In [11]:
census = pd.read_csv("output/places_totals.csv")

### Merge with our LA cases dataframe

In [12]:
places_lrg = pd.merge(places_lg, census[["name", "rsa"]], on="name", how="left")

---

### Trim the dataframe to cases during the surge

In [13]:
surge_places = places_lrg[
    (places_lrg["date"] >= surge_begin) & (places_lrg["date"] <= surge_end)
][["name", "rsa", "date", "new_confirmed_cases_seven_day_per_100k"]]

### Aggregate up to LA County as a whole

In [14]:
lacounty_all = (
    surge_places.groupby(["date"])
    .agg({"new_confirmed_cases_seven_day_per_100k": "mean"})
    .reset_index()
)

### Same for regions in LA County (Westside, Antelope Valley, etc)

In [15]:
regions = (
    surge_places.groupby(["rsa", "date"])
    .agg({"new_confirmed_cases_seven_day_per_100k": "mean"})
    .reset_index()
)

### Append LA County figures into places dataframe for context

In [16]:
lacounty_all["name"] = "Los Angeles County"

In [17]:
lacounty_all["rsa"] = "Los Angeles County"

In [18]:
surge_places = surge_places.append(lacounty_all, ignore_index=True)

In [19]:
compare_places = surge_places[
    (surge_places["name"].str.contains("Playa Del Rey"))
    #     | (surge_places["name"].str.contains("Leimert Park"))
    | (surge_places["name"] == "Los Angeles County")
]

### Chart two divergent neighborhoods

In [20]:
compare_places_chart = (
    (
        alt.Chart(compare_places)
        .mark_line()
        .encode(
            x=alt.X(
                "date:T",
                axis=alt.Axis(
                    tickCount=5,
                    format=("%b. %-d"),
                    grid=False,
                    tickColor="#ffffff",
                    title="",
                ),
            ),
            y=alt.Y(
                "new_confirmed_cases_seven_day_per_100k",
                title="",
                axis=alt.Axis(tickColor="#ffffff"),
            ),
            color=alt.Color("name:N", title=" "),
        )
        .properties(width=600, height=400, title="Smaller surge")
        .configure_axis()
    )
    .configure_view(strokeOpacity=0)
    .configure_axis(labelFontSize=13)
    .configure_legend(orient="top", symbolType="stroke")
)

In [21]:
compare_places_chart

alt.Chart(...)

In [22]:
compare_places_chart.save("visuals/compare_places_chart.png")

In [23]:
# allcities_chart = (
#     alt.Chart(surge_places.query("name != 'East Rancho Dominguez'"))
#     .mark_line()
#     .encode(
#         x=alt.X(
#             "date:T",
#             axis=alt.Axis(
#                 tickCount=4,
#                 format=("%b. %-d"),
#                 grid=False,
#                 tickColor="#ffffff",
#                 title="",
#             ),
#         ),
#         y=alt.Y(
#             "new_confirmed_cases_seven_day_per_100k",
#             title="",
#             axis=alt.Axis(tickColor="#ffffff"),
#         ),
#         facet=alt.Facet("name:N", title="", columns=11),
#         color=alt.Color("rsa:N", scale=alt.Scale(scheme="category20")),
#     )
#     .properties(width=60, height=100)
#     .configure_axis()
# )

In [24]:
# allcities_chart

In [25]:
# allcities_chart.save("visuals/allcities_chart_min_multiples.png")

### Chart cange in coronavirus rate/100k by L.A. County region

In [79]:
allregions_chart = (
    alt.Chart(regions)
    .mark_area()
    .encode(
        x=alt.X(
            "date:T",
            axis=alt.Axis(
                tickCount=4,
                format=("%b. %-d"),
                grid=False,
                tickColor="#ffffff",
                title="",
            ),
        ),
        y=alt.Y(
            "new_confirmed_cases_seven_day_per_100k",
            title="",
            axis=alt.Axis(tickColor="#ffffff"),
        ),
        facet=alt.Facet("rsa:N", title="", columns=5),
        #         color=alt.Color("rsa:N", scale=alt.Scale(scheme="category20"), title=" "),
    )
    .properties(
        width=200,
        height=200,
        title="Change in coronavirus rate/100k, by L.A. County region",
    )
    .configure_axis()
    .configure_legend(symbolType="stroke")
)

In [80]:
allregions_chart

alt.Chart(...)

In [28]:
# allcities_chart.save("visuals/allregions_chart_min_multiples.png")

---

## Change from Nov. 15 to Jan. 15?

In [29]:
surge_change = places_lrg[
    (places_lrg["date"] == surge_begin) | (places_lrg["date"] == surge_end)
][["name", "rsa", "date", "new_confirmed_cases_seven_day_per_100k"]]

### Make the change dataframe wide 

In [30]:
surge_change_pivot = (
    pd.pivot_table(
        surge_change[surge_change["new_confirmed_cases_seven_day_per_100k"] > 0],
        values="new_confirmed_cases_seven_day_per_100k",
        index=["name", "rsa"],
        columns=["date"],
    )
    .dropna()
    .reset_index()
)

### Clean up column headers, round figures and remove places with no cases

In [31]:
surge_change_pivot.rename(
    columns={"2020-11-15": "mid_november", "2021-01-15": "mid_january"}, inplace=True
)

In [32]:
surge_change_pivot[["mid_november", "mid_january"]] = round(
    surge_change_pivot[["mid_november", "mid_january"]], 2
)

In [33]:
surge_change_pivot = surge_change_pivot[surge_change_pivot["mid_november"] > 0]

### LA County increase percentage?

In [34]:
round(
    (
        (
            surge_change_pivot["mid_january"].sum()
            - surge_change_pivot["mid_november"].sum()
        )
        / surge_change_pivot["mid_november"].sum()
    )
    * 100,
    2,
)

454.57

### Calculate the change

In [35]:
surge_change_pivot["pct_change"] = round(
    (
        (surge_change_pivot["mid_january"] - surge_change_pivot["mid_november"])
        / surge_change_pivot["mid_november"]
    )
    * 100,
    2,
)

### Which places had the smallest increase in coronavirus rates? 

In [36]:
surge_change_pivot.sort_values("pct_change", ascending=True).head(10)

date,name,rsa,mid_november,mid_january,pct_change
125,Playa Del Rey,Westside,250.31,187.73,-25.00
117,Palisades Highlands,Westside,104.00,130.01,25.01
183,West Hollywood,Central L.A.,211.09,308.52,46.16
3,Agua Dulce,Northwest County,240.50,408.85,70.00
100,Malibu,Santa Monica Mountains,100.30,208.32,107.70
38,Del Aire,South Bay,91.05,227.63,150.01
47,East Pasadena,San Gabriel Valley,46.85,124.94,166.68
56,Encino,San Fernando Valley,172.67,464.89,169.24
76,Hollywood Hills,Central L.A.,139.29,397.50,185.38
131,Rancho Park,Westside,106.71,304.88,185.71


### And the greatest increase? 

In [37]:
surge_change_pivot.sort_values("pct_change", ascending=False).head(10)

date,name,rsa,mid_november,mid_january,pct_change
140,San Marino,San Gabriel Valley,15.06,316.34,2000.53
93,Leimert Park,South L.A.,59.07,774.53,1211.21
174,Vermont Square,South L.A.,156.72,1828.39,1066.66
153,South San Gabriel,San Gabriel Valley,90.42,1039.78,1049.94
78,Hyde Park,South L.A.,101.61,1152.73,1034.47
143,Santa Fe Springs,Southeast,119.80,1236.11,931.81
18,Beverlywood,Westside,53.14,538.98,914.26
10,Avocado Heights,San Gabriel Valley,118.08,1195.57,912.51
119,Palms,Westside,82.05,818.20,897.20
49,East Whittier,San Gabriel Valley,94.23,904.64,860.03


### Rank them

In [38]:
surge_change_pivot["rank_lower_is_better"] = surge_change_pivot["pct_change"].rank(
    method="max"
)

In [39]:
surge_change_pivot.sort_values("rank_lower_is_better", ascending=True).head(20)

date,name,rsa,mid_november,mid_january,pct_change,rank_lower_is_better
125,Playa Del Rey,Westside,250.31,187.73,-25.00,1.0
117,Palisades Highlands,Westside,104.00,130.01,25.01,2.0
183,West Hollywood,Central L.A.,211.09,308.52,46.16,3.0
3,Agua Dulce,Northwest County,240.50,408.85,70.00,4.0
100,Malibu,Santa Monica Mountains,100.30,208.32,107.70,5.0
38,Del Aire,South Bay,91.05,227.63,150.01,6.0
47,East Pasadena,San Gabriel Valley,46.85,124.94,166.68,7.0
56,Encino,San Fernando Valley,172.67,464.89,169.24,8.0
76,Hollywood Hills,Central L.A.,139.29,397.50,185.38,9.0
131,Rancho Park,Westside,106.71,304.88,185.71,10.0


---

### Import geography

In [40]:
places_map = gpd.read_file(
    "input/cities-neighborhoods-unincorporated-la-county.geojson"
).fillna("")

In [41]:
places_map.head()

,NAME,geometry
0,Agoura Hills,"MULTIPOLYGON (((117201.354 -426752.672, 116472..."
1,Bellflower,"MULTIPOLYGON (((174818.350 -454427.122, 174810..."
2,Beverly Hills,"MULTIPOLYGON (((148374.547 -432436.704, 148309..."
3,Burbank,"MULTIPOLYGON (((156006.516 -426897.025, 156303..."
4,Calabasas,"MULTIPOLYGON (((120408.368 -426638.916, 120344..."


In [42]:
places_map_geo = places_map.merge(
    surge_change_pivot, left_on="NAME", right_on="name", how="outer"
)

In [43]:
len(places_map_geo)

348

In [44]:
places_map_geo.head()

,NAME,geometry,name,rsa,mid_november,mid_january,pct_change,rank_lower_is_better
0,Agoura Hills,"MULTIPOLYGON (((117201.354 -426752.672, 116472...",Agoura Hills,Santa Monica Mountains,71.83,397.45,453.32,97.0
1,Bellflower,"MULTIPOLYGON (((174818.350 -454427.122, 174810...",Bellflower,Southeast,144.08,1130.76,684.81,168.0
2,Beverly Hills,"MULTIPOLYGON (((148374.547 -432436.704, 148309...",Beverly Hills,Westside,98.49,492.47,400.02,77.0
3,Burbank,"MULTIPOLYGON (((156006.516 -426897.025, 156303...",Burbank,San Fernando Valley,146.48,600.86,310.20,43.0
4,Calabasas,"MULTIPOLYGON (((120408.368 -426638.916, 120344...",Calabasas,Santa Monica Mountains,57.56,386.47,571.42,137.0


---

### Export

In [45]:
places_map_geo[
    [
        "name",
        "rsa",
        "mid_november",
        "mid_january",
        "pct_change",
        "rank_lower_is_better",
        "geometry",
    ]
].to_file("output/places_map_geo.geojson", driver="GeoJSON")

In [46]:
surge_change_pivot.rename(
    columns={
        "mid_november": "case_rate_per_100k_mid-november",
        "mid_january": "case_rate_per_100k_mid-january",
    },
    inplace=True,
)

---

### Add demographics from 07-census-data-places

In [47]:
census = pd.read_csv("output/places_totals.csv")

In [48]:
surge_demos = pd.merge(surge_change_pivot, census, on="name", how="left")

In [49]:
surge_demos.drop(["rsa_y"], axis=1, inplace=True)

In [50]:
surge_demos.rename(columns={"rsa_x": "rsa"}, inplace=True)

### Places with the HIGHEST rates of homes that have seven or more residents

In [51]:
surge_demos[
    [
        "name",
        "rsa",
        "est_total_seven_more_units_pct",
        "pct_change",
        "rank_lower_is_better",
    ]
].sort_values("est_total_seven_more_units_pct", ascending=False).head(10)

,name,rsa,est_total_seven_more_units_pct,pct_change,rank_lower_is_better
48,East Rancho Dominguez,Harbor,22.81,378.26,68.0
116,Pacoima,San Fernando Valley,21.03,396.40,74.0
185,West Puente Valley,San Gabriel Valley,19.74,825.02,181.0
7,Arleta,San Fernando Valley,18.53,374.57,66.0
10,Avocado Heights,San Gabriel Valley,18.48,912.51,188.0
177,Walnut Park,Southeast,17.62,851.63,184.0
99,Lynwood,Southeast,17.53,583.22,139.0
58,Florence-Firestone,South L.A.,17.31,528.05,123.0
178,Watts,South L.A.,17.22,752.69,177.0
149,South El Monte,San Gabriel Valley,16.74,430.19,89.0


### Places with the LOWEST rates of homes that have seven or more residents

In [52]:
surge_demos[
    [
        "name",
        "rsa",
        "est_total_seven_more_units_pct",
        "pct_change",
        "rank_lower_is_better",
    ]
].sort_values("est_total_seven_more_units_pct", ascending=True).head(10)

,name,rsa,est_total_seven_more_units_pct,pct_change,rank_lower_is_better
104,Marina del Rey,Westside,0.03,419.99,86.0
25,Carthay,Central L.A.,0.04,293.32,34.0
183,West Hollywood,Central L.A.,0.06,46.16,3.0
27,Century City,Westside,0.10,308.33,40.0
76,Hollywood Hills,Central L.A.,0.15,185.38,9.0
73,Hermosa Beach,South Bay,0.16,252.39,21.0
20,Brentwood,Westside,0.18,200.00,12.0
189,Westwood,Westside,0.25,469.07,106.0
156,Studio City,San Fernando Valley,0.26,329.98,53.0
163,Toluca Lake,San Fernando Valley,0.29,266.67,25.0


### Places with largest coronavirus rate increases during surge

In [53]:
surge_demos[
    [
        "name",
        "rsa",
        "case_rate_per_100k_mid-november",
        "case_rate_per_100k_mid-january",
        "rank_lower_is_better",
        "pct_change",
        "est_latino_pct",
        "est_total_seven_more_units_pct",
    ]
].sort_values("rank_lower_is_better", ascending=True).tail(10)

,name,rsa,case_rate_per_100k_mid-november,case_rate_per_100k_mid-january,rank_lower_is_better,pct_change,est_latino_pct,est_total_seven_more_units_pct
49,East Whittier,San Gabriel Valley,94.23,904.64,186.0,860.03,51.76,2.30
119,Palms,Westside,82.05,818.20,187.0,897.20,22.90,0.51
10,Avocado Heights,San Gabriel Valley,118.08,1195.57,188.0,912.51,85.80,18.48
18,Beverlywood,Westside,53.14,538.98,189.0,914.26,10.44,1.40
143,Santa Fe Springs,Southeast,119.80,1236.11,190.0,931.81,82.49,8.49
78,Hyde Park,South L.A.,101.61,1152.73,191.0,1034.47,36.60,4.95
153,South San Gabriel,San Gabriel Valley,90.42,1039.78,192.0,1049.94,41.25,7.32
174,Vermont Square,South L.A.,156.72,1828.39,193.0,1066.66,63.40,15.17
93,Leimert Park,South L.A.,59.07,774.53,194.0,1211.21,25.70,3.22
140,San Marino,San Gabriel Valley,15.06,316.34,195.0,2000.53,6.65,1.24


### Places with smallest coronavirus rate increases during surge

In [54]:
surge_demos[
    [
        "name",
        "rsa",
        "case_rate_per_100k_mid-november",
        "case_rate_per_100k_mid-january",
        "rank_lower_is_better",
        "pct_change",
        "est_latino_pct",
        "est_total_seven_more_units_pct",
    ]
].sort_values("rank_lower_is_better", ascending=True).head(10)

,name,rsa,case_rate_per_100k_mid-november,case_rate_per_100k_mid-january,rank_lower_is_better,pct_change,est_latino_pct,est_total_seven_more_units_pct
125,Playa Del Rey,Westside,250.31,187.73,1.0,-25.00,8.94,0.46
117,Palisades Highlands,Westside,104.00,130.01,2.0,25.01,3.90,0.54
183,West Hollywood,Central L.A.,211.09,308.52,3.0,46.16,10.50,0.06
3,Agua Dulce,Northwest County,240.50,408.85,4.0,70.00,23.23,2.19
100,Malibu,Santa Monica Mountains,100.30,208.32,5.0,107.70,6.54,0.82
38,Del Aire,South Bay,91.05,227.63,6.0,150.01,50.16,3.57
47,East Pasadena,San Gabriel Valley,46.85,124.94,7.0,166.68,26.80,2.67
56,Encino,San Fernando Valley,172.67,464.89,8.0,169.24,9.32,0.47
76,Hollywood Hills,Central L.A.,139.29,397.50,9.0,185.38,9.48,0.15
131,Rancho Park,Westside,106.71,304.88,10.0,185.71,13.51,0.47


In [55]:
surge_demos.sort_values("rank_lower_is_better", ascending=True).to_csv(
    "output/hayley/change_by_community.csv", index=False
)

### What's the relationship between our demographics and coronavirus rates? 

In [56]:
surge_demos_slim = surge_demos[
    [
        "name",
        "rsa",
        "est_pop",
        "case_rate_per_100k_mid-january",
        "pct_change",
        "est_latino_pct",
        "est_anglo_pct",
        "est_black_pct",
        "est_asian_pct",
        "est_total_seven_more_units_pct",
        "est_total_single_units_pct",
        "median_house_income",
    ]
]

In [57]:
surge_demos_slim.corr(method="pearson")

,est_pop,case_rate_per_100k_mid-january,pct_change,est_latino_pct,est_anglo_pct,est_black_pct,est_asian_pct,est_total_seven_more_units_pct,est_total_single_units_pct,median_house_income
est_pop,1.000000,0.217570,-0.048741,0.183659,-0.162911,-0.037142,-0.009635,0.089276,-0.116308,-0.169849
case_rate_per_100k_mid-january,0.217570,1.000000,0.329937,0.903094,-0.797633,0.162424,-0.319116,0.841914,-0.537789,-0.761804
pct_change,-0.048741,0.329937,1.000000,0.240216,-0.401742,0.262234,0.094233,0.258328,-0.279286,-0.314771
est_latino_pct,0.183659,0.903094,0.240216,1.000000,-0.819773,0.029543,-0.352158,0.857498,-0.599738,-0.706204
est_anglo_pct,-0.162911,-0.797633,-0.401742,-0.819773,1.000000,-0.330281,-0.075900,-0.750123,0.562412,0.801918
est_black_pct,-0.037142,0.162424,0.262234,0.029543,-0.330281,1.000000,-0.248265,0.129619,0.077794,-0.339954
est_asian_pct,-0.009635,-0.319116,0.094233,-0.352158,-0.075900,-0.248265,1.000000,-0.294317,-0.006916,0.093047
est_total_seven_more_units_pct,0.089276,0.841914,0.258328,0.857498,-0.750123,0.129619,-0.294317,1.000000,-0.674212,-0.578296
est_total_single_units_pct,-0.116308,-0.537789,-0.279286,-0.599738,0.562412,0.077794,-0.006916,-0.674212,1.000000,0.193066
median_house_income,-0.169849,-0.761804,-0.314771,-0.706204,0.801918,-0.339954,0.093047,-0.578296,0.193066,1.000000


In [58]:
scatter = (
    alt.Chart(surge_demos_slim)
    .mark_circle(size=100)
    .encode(
        x=alt.Y(
            "median_house_income:Q",
            axis=alt.Axis(grid=False, tickCount=6, format="$2,f"),
            title="Median household income",
        ),
        y=alt.Y(
            "case_rate_per_100k_mid-january:Q",
            title="Case rate per 100,000 residents — mid-January",
            axis=alt.Axis(tickColor="#ffffff"),
        ),
        color=alt.Color("rsa", scale=alt.Scale(scheme="category20")),
        size="est_pop:Q",
        tooltip=[
            "name",
            "est_pop",
            "case_rate_per_100k_mid-january",
            "est_total_seven_more_units_pct",
        ],
    )
)

text = (
    alt.Chart(surge_demos_slim)
    .mark_text(align="left", baseline="middle", dx=7, size=12)
    .encode(
        x="median_house_income:Q",
        y="case_rate_per_100k_mid-january:Q",
        text="name",
    )
)

(scatter + text).properties(width=1500, height=1000).configure_view(
    strokeOpacity=0
).configure_axis(labelFontSize=13).configure_legend()

alt.LayerChart(...)

In [59]:
(scatter + text).properties(width=2000, height=2000).save(
    "visuals/places_scatter_house_income.png"
)

In [60]:
surge_change_pivot.sort_values("rank_lower_is_better", ascending=True).head(20).to_csv(
    "output/least_affected_cities.csv", index=False
)